<a href="https://colab.research.google.com/github/byu-cce270/content/blob/main/docs/unit3/01_gspread/HW_Gspread_Starter_Sheet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Imports
#import modules here



In [ ]:
# @title Bid Analysis




In [ ]:
# @title Drywall Bid Analysis


Drywall = (sorted(Drywall, key=lambda x: x[2]))        #GIVE THIS CODE



In [ ]:
# @title Concrete Bid Analysis


Concrete = (sorted(Concrete, key=lambda x: x[2]))   #GIVE THIS CODE



In [ ]:
# @title Client Estimating Portion

myWorksheet = wb.get_worksheet(1)
values1 = myWorksheet.get_all_values()

myWorksheet = wb.get_worksheet(2)
quantities = myWorksheet.get_all_values()

def client_estimation(quantities, values1):
  # Remove the header row
  quantities.pop(0)

  # Start an empty list for the quantities and costs
  quantities_costs = []
  total_cost = 0  # Initialize total cost

  for i in quantities:
      # Calculate quantity with waste included
      quantity = float(i[1].replace(',', '')) * (1 + float(i[3].replace('%', '')) / 100)

      # Find the cost per unit from values1
      cost = None
      for j in values1:
          if j[0] == i[0]:  # Matching material name
              cost = float(j[2].replace(',', '').replace('.', ''))  # Convert to float
              break

      if cost is None:
          print(i[0], "not found in materials list")
          continue  # Skip this material if not found

      # Calculate material cost
      material_cost = quantity * cost
      total_cost += material_cost  # Accumulate total cost

      # Append
      # quantities_costs.append(f"{i[0]}: Total Cost: ${material_cost:.2f}")
      quantities_costs.append([i[0], material_cost])

  # Print results
  for i in quantities_costs: print(i)
  print("\n Total Cost: $", total_cost)
  return quantities_costs, total_cost

def update_sheet(wb, client, quantity, clientcost):
    #write code to add a sheet to your google sheet with the clients order
  worksheet = wb.add_worksheet(title = client, rows = "100", cols = "20")
  for i in quantity:
    worksheet.append_row(i)
  worksheet.append_row(["Total Cost: $", clientcost])

#find the estimate for both clients
client1_quantities, client1cost = client_estimation(quantities, values1)
update_sheet(wb, "Client1", client1_quantities, client1cost)

client2_quantities, client2cost = client_estimation(wb.get_worksheet(3).get_all_values(),values1)
update_sheet(wb, "Client2", client2_quantities, client2cost)


['Concrete', 36750.0]
['Rebar', 18540.0]
['CMU Block', 64200.0]
['Drywall', 6600.0]
['Plywood (3/4")', 9720.0]
['Lumber (2x4x8)', 23100.0]
['Structural Steel', 19584.0]
['Roofing Shingles', 3888.000000000001]
['Insulation (R-13)', 225250.0]
['Copper Wiring', 472500.0]
['PVC Piping (4")', 3090.0]
['Paint (Interior)', 2688.0000000000005]
['Paint (Exterior)', 1540.0]
['Glass Windows', 13125.0]
['Doors (Solid Core)', 6180.0]
['Tile Flooring', 192600.0]
['Carpet Flooring', 1194375.0]
['Asphalt Paving', 106000.0]

 Total Cost: $ 2399730.0
['Concrete (Footings)', 16848.0]
['Concrete (Slab)', 26460.0]
['Rebar (#5)', 11845.0]
['CMU Block (8"x8"x16")', 556500.0]
['Drywall (1/2")', 113400.00000000001]
['Plywood (OSB 5/8")', 12960.0]
['Framing Lumber (2x6x10)', 233750.0]
['Structural Steel Beams', 14688.0]
['Roofing Shingles', 5136.000000000001]
['R-19 Insulation', 322240.0]
['Electrical Conduit (1")', 331500.0]
['Copper Wiring', 708750.0]
['PVC Piping (3")', 366937.5]
['Paint (Interior)', 3024.00